In [1]:
!pip install ratsnlp

In [ ]:
# 환경설정
from ratsnlp.nlpbook.qa import QADeployArguments

args = QADeloyArguments(

    # pretrain 모델
    pretrained_model_name='beomi/kcbert-base',
    # 체크포인트 경로
    downstream_model_dir='/content/drive/MyDrive/메타버스_아카데미_2기/딥러닝/7월/models/bert/',
    # 입력 문장 최대 길이(질문,지문을 포함한 전체 글)
    max_seq_length=128,
    # 질문의 최대 길이
    max_query_length=32
)

In [ ]:
# 토크나이저

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False
)

In [ ]:
# 체크포인트 불러오기

import torch
fine_tuned_model_ckpt = torch.load(
    arg.downstream_model_checkpoint_path,
    # cpu에 모델을 불러옴
    map.location = torch.dvice('cpu')
)

In [ ]:
# Bert 설정

from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name
)

In [ ]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering(pretrained_model_config)

model.load_state_dict({k.replace('model.','') :v for k,v in fine_tuned_model_ckpt['state_dict'].items()})

In [ ]:
def inference_fn(question,context):

  if question and context:
    truncated_query = tokenizer.encode(
        question,
        add_special_token=False,
        truncation=True,
        max_length=args.max_query_length
        )
    inputs = tokenizer.encode_plus(
        text = truncated_query,
        text_pair = context,
        truncation = 'only_second',
        max_length = args.max_seq_length,
        return_token_type_ids = True

    )

    with torch.no_grad():
      outputs = model(**{k:torch.tensor([v]) for k,v in inputs.items()})
      # 정답 데이터의 시작위치, 끝위치
      start_pred = outputs.start_logits.argmax(dim=1).item()
      end_pred = outputs.end_logits.argmax(dim=1).item()
      # 정답을 decoder에 넣어 한글로만듬
      pred_text = tokenizer.decode(inputs['input_ids'])

      print("정답 : ",pred_text)